Un cop es tenen els grafs amb la forma que es desitja i en format PyTorch, es pot començar a dissenyar i entrenar la xarxa neuronal que donarà una solució al problema.

Els grafs (en format .pt) es troben a les carpetes "train_tsp_pt" i "test_tsp_pt", per a l'entrenament i per al test corresponentment.

# Importacions

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from pathlib import Path

from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GATv2Conv, LayerNorm
from torch_geometric.utils import to_dense_batch

# Càrrega de fitxers

In [27]:
train_path = Path("Datasets/train_pt")
graphs_train = [torch.load(f) for f in sorted(train_path.glob("*.pt"))]
print(f"Nombre de grafs de train: {len(graphs_train)}")

/tmp/ipykernel_3464/4247743316.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graphs_train = [torch.load(f) for f in sorted(train_path.glob("*.pt"))]


Nombre de grafs de train: 1514


In [28]:
# Exemples
data = graphs_train[300]
print(data)
print(data.x)
print(data.y)

Data(x=[27, 2], edge_index=[2, 351], edge_attr=[351, 1], y=26, id=[27])
tensor([[1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 1.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
tensor(26)


In [29]:
# Normalitzar valors arestes
#for g in graphs_train + graphs_test:
#    mean, std = g.edge_attr.mean(), g.edge_attr.std()
#    g.edge_attr = (g.edge_attr - mean) / (std + 1e-8)

In [30]:
class TSPGraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]

train_dataset = TSPGraphDataset(graphs_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Disseny GNN

In [ ]:
class TSPGNN(nn.Module):
    def __init__(self, in_channels=2, hidden_channels=64, heads=4, num_layers=4):
        super().__init__()

        self.input_proj = nn.Linear(in_channels, hidden_channels * heads)

        self.layers = nn.ModuleList()
        self.norms = nn.ModuleList()

        # First layer
        self.layers.append(GATv2Conv(hidden_channels * heads, hidden_channels, heads=heads, edge_dim=1))
        self.norms.append(LayerNorm(hidden_channels * heads))

        # Hidden layers
        for _ in range(num_layers - 1):
            self.layers.append(GATv2Conv(hidden_channels * heads, hidden_channels, heads=heads, edge_dim=1))
            self.norms.append(LayerNorm(hidden_channels * heads))

        # Output per node (score per node)
        self.out = nn.Linear(hidden_channels * heads, 1)

    def forward(self, data, return_probs=False):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        edge_attr = edge_attr.view(-1, 1)  # assegura la forma correcta

        # Map input a hidden
        x = self.input_proj(x)

        # Residual connections
        for conv, norm in zip(self.layers, self.norms):
            h = conv(x, edge_index, edge_attr)
            h = norm(h)
            h = F.relu(h)
            x = x + h  # residual safe, dimensions coincideixen

        logits = self.out(x).squeeze(-1)  # [num_nodes_total]

        if return_probs:
            # Probabilitats per node dins cada graf
            x_dense, mask = to_dense_batch(logits.unsqueeze(-1), batch=data.batch)
            probs = torch.softmax(x_dense, dim=1)  # softmax per nodes dins el graf
            return probs, mask

        return logits

# Training

## Configuració

In [ ]:
# --- CONFIGURACIÓ ---
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TSPGNN(
    in_channels=2,
    hidden_channels=32,
    heads=4,
    num_layers=2
).to(device)

# Optimitzador amb regularització
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=0.001,
    weight_decay=1e-4
)

# Scheduler 
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=20,
    gamma=0.5
)

# Classificació per node, y = target_idx (enter)
criterion = nn.CrossEntropyLoss()

## Funcions

In [34]:
def train(model, loader, optimizer, criterion, device, scheduler=None):
    model.train()
    total_loss = 0.0

    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()

        # logits per node
        logits = model(data)  # [num_nodes_total]

        # Transformem a batch dens per graf
        logits_dense, mask = to_dense_batch(logits.unsqueeze(-1), batch=data.batch)
        # logits_dense: [batch_size, max_num_nodes, 1]
        # mask: [batch_size, max_num_nodes] → True per nodes reals

        batch_loss = 0.0
        batch_size = logits_dense.size(0)

        for i in range(batch_size):
            num_nodes_i = mask[i].sum()
            logits_i = logits_dense[i, :num_nodes_i, 0]  # nodes reals
            target_i = data.y[i]  # enter que indica el target dins del graf
            batch_loss += criterion(logits_i.unsqueeze(0), target_i.unsqueeze(0))

        batch_loss /= batch_size
        batch_loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 2.0)
        optimizer.step()

        if scheduler is not None:
            scheduler.step()

        total_loss += batch_loss.item()

    return total_loss / len(loader)

In [35]:
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    total_correct_top1 = 0
    total_graphs = 0
    normalized_ranks = []

    for data in loader:
        data = data.to(device)
        # logits dens per graf i softmax per node
        probs_dense, mask = model(data, return_probs=True)
        batch_size, max_nodes, _ = probs_dense.size()

        for i in range(batch_size):
            num_nodes_i = mask[i].sum()
            probs_i = probs_dense[i, :num_nodes_i, 0]  # nodes reals
            target_i = data.y[i]

            # --- Top1 Accuracy ---
            pred_idx = probs_i.argmax()
            if pred_idx == target_i:
                total_correct_top1 += 1
            total_graphs += 1

            # --- Normalized Rank ---
            sorted_indices = torch.argsort(probs_i, descending=True)
            rank = (sorted_indices == target_i).nonzero(as_tuple=True)[0].item() + 1
            normalized_rank = (rank - 1) / (num_nodes_i - 1)
            normalized_ranks.append(normalized_rank)

    top1_acc = total_correct_top1 / total_graphs
    mean_normalized_rank = sum(normalized_ranks) / len(normalized_ranks)

    return top1_acc, mean_normalized_rank

## Entrenament

In [36]:
# --- ENTRENAMENT ---
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    loss = train(model, train_loader, optimizer, criterion, device, scheduler=scheduler)
    top1_acc, mean_rank = evaluate(model, train_loader, device)
    print(f"Epoch {epoch:02d} | Loss: {loss:.3f} | Top1Acc: {top1_acc:.3f} | MeanRank: {mean_rank:.3f}")

Epoch 01 | Loss: 3.245 | Top1Acc: 0.342 | MeanRank: 0.234
Epoch 02 | Loss: 2.940 | Top1Acc: 0.355 | MeanRank: 0.211
Epoch 03 | Loss: 2.907 | Top1Acc: 0.362 | MeanRank: 0.199
Epoch 04 | Loss: 2.881 | Top1Acc: 0.365 | MeanRank: 0.199
Epoch 05 | Loss: 2.898 | Top1Acc: 0.365 | MeanRank: 0.199
Epoch 06 | Loss: 2.908 | Top1Acc: 0.365 | MeanRank: 0.199
Epoch 07 | Loss: 2.896 | Top1Acc: 0.365 | MeanRank: 0.198
Epoch 08 | Loss: 2.896 | Top1Acc: 0.364 | MeanRank: 0.199
Epoch 09 | Loss: 2.882 | Top1Acc: 0.363 | MeanRank: 0.198
Epoch 10 | Loss: 2.903 | Top1Acc: 0.363 | MeanRank: 0.198


In [37]:
# --- GUARDAR MODEL ---
save_path = "model_gnn2.pt"
torch.save(model.state_dict(), save_path)
print(f"Model guardat a {save_path}")

Model guardat a model_gnn2.pt


In [40]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())


2.4.1+cu121
12.1
True
